# Data Cleaning -  USD/EUR 40years

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql
import import_ipynb # pip install import-ipynb
#from toolbox import * # helper functions 

#### Raw Price Data 

In [2]:
l = []
for f in range(1,4):
    l.append(pd.read_csv("usd_eur_40yrs/%s.csv"%(f),
                     na_filter=True,
                     parse_dates=True,
                     index_col="Date",
                    ))
# merge 10year csvs
df = pd.concat(l,axis=0)
# sort by date
df.sort_values(["Date"],inplace=True)
# needless feature
df.drop("Change %",axis=1,inplace=True)
print (df.shape)
df.head()

(7672, 4)


Price    Open    High     Low
Date                                      
1980-01-02  0.6601  0.6601  0.6601  0.6601
1980-01-03  0.6589  0.6589  0.6589  0.6589
1980-01-04  0.6610  0.6610  0.6610  0.6610
1980-01-07  0.6593  0.6593  0.6593  0.6593
1980-01-08  0.6619  0.6619  0.6619  0.6619

In [3]:
# Save to database
conn = sql.connect('database.db')
cur = conn.cursor()
df.to_sql("raw_prices", conn, if_exists="replace")
conn.close()

## Section for TA indicators (Future)

- No volume in dataset

## Create GAF images
- apply a 5day rolling window to create 1day step labels for supervised learning 
- independent variables for training set:  Open, High, Low
- dependent variable: Price
- 80-20 train/test split
- **Just 2 classes B/S** 

In [4]:
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField # conda install -c conda-forge pyts

# Parameters
path = 'database.db'
table_name = 'raw_prices'
n_steps = 5 # 5 day rolling window 
split_threshold = 0.80 # 80-20 train-test split
transaction_threshold = 1 # Transaction cost per trade as a % ( 1 = 1 %)

In [5]:
# Retrieve raw prices
conn = sql.connect(path)
cur = conn.cursor()
df = pd.read_sql_query("SELECT Date,Price, Open, High, Low FROM %s" %(table_name), conn)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'],inplace=True)

# Create labels for supervised learning
df_classes = pd.DataFrame()
df_classes["t"] = df['Price']
df_classes["t+1"] = df['Price'].shift(1); df_classes["t+1"][0] = 0.6601 # fill original val
df_classes["class"] = 0
df_classes["class"].where(df_classes["t"].values>df_classes["t+1"].values,
                          1, # if price rise -> BUY , else -> SELL
                          inplace=True,  )
print(df_classes.head(2))
# Save labels to database
df_classes.to_sql("classes", conn, if_exists="replace")
conn.close()

# Set as new feature in df
df['Class'] = df_classes["class"].values

                 t     t+1  class
Date                             
1980-01-02  0.6601  0.6601      1
1980-01-03  0.6589  0.6601      1


In [7]:
labels = pd.DataFrame(df['Class'].iloc[::5]) # every 5th value is a label


Class
Date             
1980-01-02      1
1980-01-09      1
1980-01-16      1
1980-01-23      1
1980-01-30      1
...           ...
2009-12-03      1
2009-12-10      1
2009-12-17      0
2009-12-24      1
2009-12-31      0

[1535 rows x 1 columns]

In [8]:
# WORKS but need to do on train / test sets - this is everything

train_d = df.drop(["Price","Class"],axis=1).to_numpy()
l = []
for i in range(len(train_d)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(train_d)-1:
        break
    l.append(train_d[i:end_ix, :])
np.array(l).shape    

(7667, 5, 3)

In [ ]:
        def split_time_series(dfx):
            dfx_train = dfx.loc[:, dfx.columns != 'Price']
            dfx_train = dfx_train.to_numpy()
            dfx_labels = dfx['Price']
            dfx_labels = dfx_labels.to_numpy()
            X, y = list(), list()
            for i in range(len(dfx_train)):
                # find the end of this pattern
                end_ix = i + self.n_steps
                # check if we are beyond the dataset
                if end_ix > len(dfx_train)-1:
                    break
                # gather input and output parts of the pattern
                seq_x, seq_y = dfx_train[i:end_ix, :], dfx_labels[end_ix] 
                X.append(seq_x)
                y.append(seq_y)

In [ ]:
data = pre_processing(df, n_steps , split_threshold)

In [ ]:
from numpy import array

train_data, test_data = data.split_data()

print(train_data.shape, test_data.shape )
# TRANSFROM ALL FEATURES EXCEPT ADJCLOSE PRICES
tranformed_train, transformed_test = scale_and_polarise(train_data.iloc[:,:-1],test_data.iloc[:,:-1])
tranformed_train.columns, transformed_test.columns = train_data.drop(['Price'],axis=1).columns, test_data.drop(['Price'],axis=1).columns

tranformed_train['Price'] = train_data.iloc[:,-1].values
transformed_test['Price'] = test_data.iloc[:,-1].values

tranformed_train.index, transformed_test.index = train_data.index, test_data.index
tX, tY, vX, vY  = data.tensor_samples_labels(tranformed_train, transformed_test, table_name)

In [ ]:
tX.shape, vX.shape

In [ ]:
tY

In [ ]:
def scale_and_polarise(train_data,test_data):
    # def prep_data
    " Scale data between [ 0 , 1 ] with MinMax  "
    from sklearn.preprocessing import MinMaxScaler
    scaler1 = MinMaxScaler()
    scaler2 = MinMaxScaler()

    scaled_train = scaler1.fit_transform(train_data)
    scaled_test= scaler2.fit_transform(test_data)
    #print(np.max(scaled_train) , np.max(scaled_test)) # sanity check

    " Encode angles as Polar Coordinates  "
    polar_train = np.arccos(scaled_train)
    polar_test = np.arccos(scaled_test)
    #print(np.max(polar_train) , np.max(polar_test)) # sanity check

    return pd.DataFrame(polar_train), pd.DataFrame(polar_test) 

In [ ]:
class pre_processing ():
    def __init__(self, df, n_steps , split_threshold):
        self.df = df
        self.n_steps = n_steps # the size of the sliding window
        self.split_threshold = split_threshold  # the ratio for the train:test split
        self.df_train = None
        self.df_test = None
    def split_data(self):
        # 80 - 20 data split 
        self.df_train = self.df.iloc[ : int(len(self.df)*split_threshold), : ]
        self.df_test = self.df.iloc[ int(len(self.df)*split_threshold):, : ]         
        return self.df_train, self.df_test        
    def tensor_samples_labels (self,df_train,df_test,table):
        # create training data and labels for the time_series
        def split_time_series(dfx):
            dfx_train = dfx.loc[:, dfx.columns != 'Price']
            dfx_train = dfx_train.to_numpy()
            dfx_labels = dfx['Price']
            dfx_labels = dfx_labels.to_numpy()
            X, y = list(), list()
            for i in range(len(dfx_train)):
                # find the end of this pattern
                end_ix = i + self.n_steps
                # check if we are beyond the dataset
                if end_ix > len(dfx_train)-1:
                    break
                # gather input and output parts of the pattern
                seq_x, seq_y = dfx_train[i:end_ix, :], dfx_labels[end_ix] 
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y) 
        # convert into input/output
        train_X, train_Y = split_time_series(df_train)
        val_X, val_Y = split_time_series(df_test)
        return train_X, train_Y , val_X, val_Y  